In [ ]:
import notebookutils
import pandas as pd
import datetime
import re,json
import sempy
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
from pyspark.sql import DataFrame
from pyspark.sql.functions import col,current_timestamp,lit

# instantiate the Fabric rest client
client = fabric.FabricRestClient()

# get the current workspace ID based on the context of where this notebook is run from
thisWsId = notebookutils.runtime.context['currentWorkspaceId'] 

# this function attempts to save a dataframe which can be either a pandas dataframe or spark dataframe
def saveTable(pdf,table_name, mode='overwrite'):
    if mode=='append' and not any(table.name == table_name for table in spark.catalog.listTables()):
            mode = 'overwrite'

    if (isinstance(pdf, pd.DataFrame) and pdf.empty) or \
       (isinstance(pdf, DataFrame) and pdf.isEmpty()):
        return('No ' + table_name + ' found, nothing to save (Dataframe is empty)')
    if not isinstance(pdf, DataFrame):
        pdf = spark.createDataFrame(pdf)

    df = pdf.select([col(c).alias(
            c.replace( '(', '')
            .replace( ')', '')
            .replace( ',', '')
            .replace( ';', '')
            .replace( '{', '')
            .replace( '}', '')
            .replace( '\n', '')
            .replace( '\t', '')
            .replace( ' ', '_')
            .replace( '.', '_')
        ) for c in pdf.columns])
    #display(df)
    df.withColumn("metaTimestamp",current_timestamp()).write.mode(mode) \
      .option("mergeSchema", "true").saveAsTable(table_name)
    return(str(df.count()) +' records saved to the '+table_name + ' table.')

def saveCapacityMeta():
    spark.sql("drop table if exists capacitiess")
    df = fabric.list_capacities()
    print(saveTable(df,"capacities"))

def saveWorkspaceMeta(suppress_output=False):
    spark.sql("drop table if exists workspaces")
    df = fabric.list_workspaces()
    #display(df)
    if not suppress_output:
        print(saveTable(df,"workspaces"))
    else:
        saveTable(df,"workspaces")

def saveItemMeta(verbose_logging,ws_ignore_list, ws_ignore_like_list):
    all_items = []
    spark.sql("drop table if exists items")
    saveResults = saveWorkspaceMeta()

    wsitemssql = "SELECT distinct ID,Type,Name FROM workspaces where 1=1"

    if len(ws_ignore_like_list)>0:
        for notlike in ws_ignore_like_list:
            wsitemssql  = wsitemssql + " and name not like '" + notlike + "'"

    if len(ws_ignore_list)>0:
        wsitemssql  = wsitemssql + " and name not in ('" + "', '".join(ws_ignore_list)+ "') "
    df = spark.sql(wsitemssql).collect()

    for i in df:
        if verbose_logging:
            print('Getting items for workspace ' + i['Name'] + '...')
        if i['Type'] == 'Workspace':
            url = "/v1/workspaces/" + i['ID'] + "/items"
        try:
            itmresponse = client.get(url)
            #print(itmresponse.json()) 
            all_items.extend(itmresponse.json()['value']) 
        except Exception as error:
            errmsg =  "Couldn't get list of items for workspace " + i['Name'] + "("+ i['ID'] + ")."
            if (verbose):
                 errmsg = errmsg + "Error: "+str(error)
            print(str(errmsg))
    itmdf=spark.read.json(sc.parallelize(all_items))
    print(saveTable(itmdf,'items'))

def saveReportMeta(verbose_logging, only_secondary=False,secondary_ws_suffix='', ws_ignore_list=[], ws_ignore_like_list=[]):
    all_report_data = []
    table_name = 'reports'
    spark.sql("Drop table if exists "+ table_name)
    reportsql = "SELECT distinct ID,Type,Name FROM workspaces where Type!='AdminInsights'"
    
    if len(ws_ignore_like_list)>0:
        for notlike in ws_ignore_like_list:
            reportsql  = reportsql + " and name not like '" + notlike + "'"
    if len(ws_ignore_list)>0:
        reportsql  = reportsql + " and name not in ('" + "', '".join(ws_ignore_list)+ "') "

    if only_secondary:
        reportsql = reportsql + " and Name like '%"+secondary_ws_suffix+"'" 
    reportdf = spark.sql(reportsql).collect()

    for idx,i in enumerate(reportdf):
        if i['Type'] == 'Workspace':
            try:
                if verbose_logging:
                    print('Retreiving reports for workspace '+ i['Name'] + '...')
                dfwsreports = fabric.list_reports(i['ID'] )   
                if idx == 0:
                        dfallreports = dfwsreports
                else:
                        dfallreports = pd.concat([dfallreports, dfwsreports], ignore_index=True, sort=False)
            except WorkspaceNotFoundException as e:
                print("WorkspaceNotFoundException:", e)
            except FabricHTTPException as e:
                print("Caught a FabricHTTPException. Check the API endpoint, authentication.")

            except Exception as error:
                errmsg =  "Couldn't retreive report details for workspace " + i['Name'] + "("+ i['ID'] + "). Error: "+str(error)
                print(str(errmsg))

    dfallreports = dfallreports.drop('Subscriptions', axis=1)
    dfallreports = dfallreports.drop('Users', axis=1)
    print(saveTable(dfallreports,'reports'))

def get_lh_object_list(base_path,data_types = ['Tables', 'Files'])->pd.DataFrame:

    '''
    Function to get a list of tables for a lakehouse
    adapted from https://fabric.guru/getting-a-list-of-folders-and-delta-tables-in-the-fabric-lakehouse
    This function will return a pandas dataframe containing names and abfss paths of each folder for Files and Tables
    '''
    #data_types = ['Tables', 'Files'] #for if you want a list of files and tables
    #data_types = ['Tables'] #for if you want a list of tables

    df = pd.concat([
        pd.DataFrame({
            'name': [item.name for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
            'type': data_type[:-1].lower() , 
            'src_path': [item.path for item in notebookutils.fs.ls(f'{base_path}/{data_type}/')],
        }) for data_type in data_types], ignore_index=True)

    return df

def get_wh_object_list(schema_list,base_path)->pd.DataFrame:

    '''
    Function to get a list of tables for a warehouse by schema
    '''
    data_type = 'Tables'
    df = pd.concat([pd.DataFrame({
            'schema': schema_prefix,
            'name': [item.name for item in notebookutils.fs.ls(f'{base_path}/{data_type}/{schema_prefix}/')],
            'type': data_type[:-1].lower() , 
            'src_path': [item.path for item in notebookutils.fs.ls(f'{base_path}/{data_type}/{schema_prefix}/')],
        }) for schema_prefix in schema_list], ignore_index=True)
    return df

def copy_lh_objects(table_list,workspace_src,workspace_tgt,lakehouse_src,lakehouse_tgt,recovered_object_suffix,fastcopy=True,usingIDs=False)->pd.DataFrame:
    # declare an array to keep the instrumentation
    cpresult = []
    # loop through all the tables to extract the source path 
    for table in table_list.src_path:
        source = table
        destination = source.replace(f'abfss://{workspace_src}', f'abfss://{workspace_tgt}')
        if usingIDs:
            destination = destination.replace(f'{lakehouse_src}', f'{lakehouse_tgt}')
        else:
            destination = destination.replace(f'{lakehouse_src}.Lakehouse', f'{lakehouse_tgt}.Lakehouse') + recovered_object_suffix
        start_time =  datetime.datetime.now()
        if notebookutils.fs.exists(destination):
             notebookutils.fs.rm(destination, True)
        if fastcopy:
            # use fastcopy util which is a python wrapper to azcopy
            notebookutils.fs.fastcp(source+'/*', destination+'/', True)
        else:
            notebookutils.fs.cp(source, destination, True)

        # recording the timing and add it to the results list
        end_time = datetime.datetime.now()
        copyreslist = [source, destination, start_time.strftime("%Y-%m-%d %H:%M:%S"),  end_time.strftime("%Y-%m-%d %H:%M:%S"), str((end_time - start_time).total_seconds())]
        cpresult.append(copyreslist)
    return pd.DataFrame(cpresult,columns =['source--------------------------------------','target--------------------------------------','start------------','end_time------------','elapsed seconds----'])

def getItemId(wks_id,itm_name,itm_type):
    df = fabric.list_items(type=None,workspace=wks_id)
    #print(df)
    if df.empty:
        #print('No items found in workspace '+i['Name']+' (Dataframe is empty)')
        return 'NotExists'
    else:
        #display(df)
        #print(df.query('"Display Name"="'+itm_name+'"'))
        if itm_type != '':
          newdf= df.loc[(df['Display Name'] == itm_name) & (df['Type'] == itm_type)]['Id']
        else:
          newdf= df.loc[(df['Display Name'] == itm_name)]['Id']  
        if newdf.empty:
          return 'NotExists'
        else:
          return newdf.iloc[0]

###### Function to create DW recovery pipeline
def createDWrecoverypl(ws_id,pl_name = 'Recover_Warehouse_Data_From_DR'):
  client = fabric.FabricRestClient()

  dfurl = "v1/workspaces/"+ ws_id + "/items"
  payload = { 
  "displayName": pl_name, 
  "type": "DataPipeline", 
  "definition": { 
    "parts": [ 
      { 
        "path": "pipeline-content.json", 
        "payload": "ewogICAgInByb3BlcnRpZXMiOiB7CiAgICAgICAgImFjdGl2aXRpZXMiOiBbCiAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICJuYW1lIjogIkl0ZXJhdGVTY2hlbWFUYWJsZXMiLAogICAgICAgICAgICAgICAgInR5cGUiOiAiRm9yRWFjaCIsCiAgICAgICAgICAgICAgICAiZGVwZW5kc09uIjogW10sCiAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgIml0ZW1zIjogewogICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy50YWJsZXNUb0NvcHkiLAogICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgImJhdGNoQ291bnQiOiAyMCwKICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdGllcyI6IFsKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiQ29weVdhcmVob3VzZVRhYmxlcyIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJDb3B5IiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJkZXBlbmRzT24iOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYWN0aXZpdHkiOiAiU2V0IHRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImRlcGVuZGVuY3lDb25kaXRpb25zIjogWwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIlN1Y2NlZWRlZCIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgXQogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIF0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAicG9saWN5IjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0aW1lb3V0IjogIjAuMTI6MDA6MDAiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJyZXRyeSI6IDIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInJldHJ5SW50ZXJ2YWxJblNlY29uZHMiOiAzMCwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlT3V0cHV0IjogZmFsc2UsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZUlucHV0IjogZmFsc2UKICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNvdXJjZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiTGFrZWhvdXNlVGFibGVTb3VyY2UiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGF0YXNldFNldHRpbmdzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibGlua2VkU2VydmljZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICI4Yzc1MTJjZF8wN2VhXzRkMzVfYjYzNl9lMDQwZTI0YjQxYTQiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiYW5ub3RhdGlvbnMiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiTGFrZWhvdXNlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIndvcmtzcGFjZUlkIjogIkBwaXBlbGluZSgpLnBhcmFtZXRlcnMud29ya3NwYWNlSWQiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFydGlmYWN0SWQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy5sYWtlaG91c2VJZCIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAicm9vdEZvbGRlciI6ICJUYWJsZXMiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiTGFrZWhvdXNlVGFibGUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidGFibGUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAY29uY2F0KGNvbmNhdChpdGVtKCkuc2NoZW1hLCdfJyksaXRlbSgpLm5hbWUpIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRXhwcmVzc2lvbiIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzaW5rIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJEYXRhV2FyZWhvdXNlU2luayIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhbGxvd0NvcHlDb21tYW5kIjogdHJ1ZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImRhdGFzZXRTZXR0aW5ncyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhbm5vdGF0aW9ucyI6IFtdLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImxpbmtlZFNlcnZpY2UiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiMGMwMzEyM2FfZDMxMl80NmM0X2E4ZTdfNWI0Y2FkOGYxMmQ3IiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAicHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFubm90YXRpb25zIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlIjogIkRhdGFXYXJlaG91c2UiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZVByb3BlcnRpZXMiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZW5kcG9pbnQiOiAiQHBpcGVsaW5lKCkucGFyYW1ldGVycy5jb25uU3RyIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhcnRpZmFjdElkIjogIkBwaXBlbGluZSgpLnBhcmFtZXRlcnMud2FyZWhvdXNlSWQiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIndvcmtzcGFjZUlkIjogIkBwaXBlbGluZSgpLnBhcmFtZXRlcnMud29ya3NwYWNlSWQiCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRGF0YVdhcmVob3VzZVRhYmxlIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiBbXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQGl0ZW0oKS5zY2hlbWEiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInRhYmxlIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiAiQGl0ZW0oKS5uYW1lIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRXhwcmVzc2lvbiIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJlbmFibGVTdGFnaW5nIjogdHJ1ZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHJhbnNsYXRvciI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiVGFidWxhclRyYW5zbGF0b3IiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZUNvbnZlcnNpb24iOiB0cnVlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZUNvbnZlcnNpb25TZXR0aW5ncyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJhbGxvd0RhdGFUcnVuY2F0aW9uIjogdHJ1ZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0cmVhdEJvb2xlYW5Bc051bWJlciI6IGZhbHNlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogIlNldCB0YWJsZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJTZXRWYXJpYWJsZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGVwZW5kc09uIjogWwogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgImFjdGl2aXR5IjogIlNldCBzY2hlbWEiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGVwZW5kZW5jeUNvbmRpdGlvbnMiOiBbCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAiU3VjY2VlZGVkIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICBdCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgfQogICAgICAgICAgICAgICAgICAgICAgICAgICAgXSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJwb2xpY3kiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInNlY3VyZU91dHB1dCI6IGZhbHNlLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzZWN1cmVJbnB1dCI6IGZhbHNlCiAgICAgICAgICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGVQcm9wZXJ0aWVzIjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YXJpYWJsZU5hbWUiOiAiVGFibGVuYW1lIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFsdWUiOiB7CiAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6ICJAaXRlbSgpLm5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJFeHByZXNzaW9uIgogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiU2V0IHNjaGVtYSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAidHlwZSI6ICJTZXRWYXJpYWJsZSIsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAiZGVwZW5kc09uIjogW10sCiAgICAgICAgICAgICAgICAgICAgICAgICAgICAicG9saWN5IjogewogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJzZWN1cmVPdXRwdXQiOiBmYWxzZSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAic2VjdXJlSW5wdXQiOiBmYWxzZQogICAgICAgICAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ0eXBlUHJvcGVydGllcyI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAidmFyaWFibGVOYW1lIjogIlNjaGVtYW5hbWUiLAogICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICJ2YWx1ZSI6IHsKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInZhbHVlIjogIkBpdGVtKCkuc2NoZW1hIiwKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgInR5cGUiOiAiRXhwcmVzc2lvbiIKICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICAgICAgICAgIH0KICAgICAgICAgICAgICAgICAgICBdCiAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgIH0KICAgICAgICBdLAogICAgICAgICJwYXJhbWV0ZXJzIjogewogICAgICAgICAgICAibGFrZWhvdXNlSWQiOiB7CiAgICAgICAgICAgICAgICAidHlwZSI6ICJzdHJpbmciLAogICAgICAgICAgICAgICAgImRlZmF1bHRWYWx1ZSI6ICI3OTUyMTZjMC0zY2EwLTQ4NGUtYTk0MC0wNWFkMTNkMTFlYzUiCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJ0YWJsZXNUb0NvcHkiOiB7CiAgICAgICAgICAgICAgICAidHlwZSI6ICJhcnJheSIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogWwogICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6ICJkYm8iLAogICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJkaW1lbnNpb25fY2l0eSIKICAgICAgICAgICAgICAgICAgICB9LAogICAgICAgICAgICAgICAgICAgIHsKICAgICAgICAgICAgICAgICAgICAgICAgInNjaGVtYSI6ICJkYm8iLAogICAgICAgICAgICAgICAgICAgICAgICAibmFtZSI6ICJkaW1lbnNpb25fY3VzdG9tZXIiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiZGltZW5zaW9uX2RhdGUiCiAgICAgICAgICAgICAgICAgICAgfSwKICAgICAgICAgICAgICAgICAgICB7CiAgICAgICAgICAgICAgICAgICAgICAgICJzY2hlbWEiOiAiZGJvIiwKICAgICAgICAgICAgICAgICAgICAgICAgIm5hbWUiOiAiZGltZW5zaW9uX2VtcGxveWVlIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogImRpbWVuc2lvbl9zdG9ja19pdGVtIgogICAgICAgICAgICAgICAgICAgIH0sCiAgICAgICAgICAgICAgICAgICAgewogICAgICAgICAgICAgICAgICAgICAgICAic2NoZW1hIjogImRibyIsCiAgICAgICAgICAgICAgICAgICAgICAgICJuYW1lIjogImZhY3Rfc2FsZSIKICAgICAgICAgICAgICAgICAgICB9CiAgICAgICAgICAgICAgICBdCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJ3b3Jrc3BhY2VJZCI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogIjRjYTI4YWYzLTE1MTAtNGNhMy04YWMxLTJmMzgzNGJiNzE1ZiIKICAgICAgICAgICAgfSwKICAgICAgICAgICAgIndhcmVob3VzZUlkIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAic3RyaW5nIiwKICAgICAgICAgICAgICAgICJkZWZhdWx0VmFsdWUiOiAiNTk0OTFlYWEtZWQ5Zi00ZDAzLWI2ZjctOGE2NGQ4YzFlMzY4IgogICAgICAgICAgICB9LAogICAgICAgICAgICAiY29ublN0ciI6IHsKICAgICAgICAgICAgICAgICJ0eXBlIjogInN0cmluZyIsCiAgICAgICAgICAgICAgICAiZGVmYXVsdFZhbHVlIjogImxubG94aGprZXEzZTNuc3Zmc3RsZGhsNG1xLTZvZmtldGFxY3dydXpjd2JmNDRkam8zcmw0LmRhdGF3YXJlaG91c2UuZmFicmljLm1pY3Jvc29mdC5jb20iCiAgICAgICAgICAgIH0KICAgICAgICB9LAogICAgICAgICJ2YXJpYWJsZXMiOiB7CiAgICAgICAgICAgICJUYWJsZW5hbWUiOiB7CiAgICAgICAgICAgICAgICAidHlwZSI6ICJTdHJpbmciCiAgICAgICAgICAgIH0sCiAgICAgICAgICAgICJTY2hlbWFuYW1lIjogewogICAgICAgICAgICAgICAgInR5cGUiOiAiU3RyaW5nIgogICAgICAgICAgICB9CiAgICAgICAgfSwKICAgICAgICAibGFzdE1vZGlmaWVkQnlPYmplY3RJZCI6ICJhN2IzMjgyNy1lNTFmLTQ0ZmYtOGZjZC0yYTJlMzZlMjVjYTEiLAogICAgICAgICJsYXN0UHVibGlzaFRpbWUiOiAiMjAyNC0wOC0yNlQxMzo1NTowNVoiCiAgICB9Cn0=", 
        "payloadType": "InlineBase64" 
      } 
    ] 
  } 
}   
  
  response = json.loads(client.post(dfurl,json= payload).content)
  return response['id']

def getWorkspaceRolesAssignments(pWorkspaceId):
    url = "/v1/workspaces/" +pWorkspaceId + "/roleAssignments"
    try:
        print('Attempting to connect workspace '+ i['Workspace_Name'])
        response = client.post(url,json= json.loads(payload))
        print(str(response.status_code) + response.text) 
        success = True
    except Exception as error:
        errmsg =  "Couldn't connect git to workspace " + i['Workspace_Name'] + "("+ i['Workspace_ID'] + "). Error: "+str(error)
        print(str(errmsg))


class noDefaultLakehouseException(Exception):
    pass